In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pennylane as qml
from matplotlib import pyplot as plt
from pennylane import numpy as np
import scipy
import networkx as nx
import copy

In [3]:
weighted_graph_adj_matrix = np.array([
    [0., 1., 2., 1.],
    [1., 0., 3., 2.],
    [2., 3., 0., 1.],
    [1., 2., 1., 0.]
])

In [77]:
from qgscnn import CouplingGate


In [78]:
def compute_L_matrix(adj_matrix):
    vertex_sums = np.sum(adj_matrix, axis=0)
    out = np.zeros(adj_matrix.shape)
    for j in range(len(adj_matrix)):
        for k in range(len(adj_matrix)):
            if j != k:
                out[j][k] = -1. * adj_matrix[j][k]
            else:
                out[j][k] = vertex_sums[j] - adj_matrix[j][k]
                
    return out
        

In [79]:
def target_hamiltonian(adj_matrix):
    L_matrix = compute_L_matrix(adj_matrix)
    out = np.zeros((4,4))
    for i in range(len(L_matrix)):
        for j in range(len(L_matrix)):
            outer_prod_i = np.array([
                [0.,0.],
                [0., 1.]
            ])
            
            outer_prod_j = np.array([
                [0., 0.],
                [0., 1.]
            ])
            
            gate_matrix = L_matrix[i][j] * np.kron(outer_prod_i, outer_prod_j)
    return out + np.identity(len(adj_matrix))

In [80]:
def coupling_sublayer(L_matrix, weights, wires):
    for i in range(len(L_matrix)):
        for j in range(len(L_matrix)):
            outer_prod_i = np.array([
                [1.-np.exp(-1j*weights[j]/2),0.],
                [0., 1.-np.exp(-1j*weights[j]/2)]
            ])

            outer_prod_j = np.array([
                [1.-np.exp(-1j*weights[j]/2), 0.],
                [0., 1.+np.exp(-1j*weights[j]/2)]
            ])

            gate_matrix = L_matrix[i][j] * np.kron(outer_prod_i, outer_prod_j)

            CouplingGate(gate_matrix, weights[i], weights[j], wires=list(set([i,j])))
    

In [93]:
def qgscnn_layer(weights, wires, L_matrix):
    # add the coupling gates
    #coupling_sublayer(L_matrix, weights[0], wires)
    for i in range(len(L_matrix)):
        for j in range(len(L_matrix)):
            if i != j:
                if L_matrix[i][j] != 0.:
                    qml.CNOT(wires=[i,j])

    # add X rotations for the kinetic hamiltonian
    for i in range(len(L_matrix)):
        qml.RX(weights[0][i], wires=i)
    

In [96]:
def quantum_clustering(adj_matrix):

    # the quantum device
    dev = qml.device("default.qubit", wires=4)
    
    # compute the L matrix
    L_matrix = compute_L_matrix(adj_matrix)
    
    # helps the cost fn if we declare how many vertices graph has
    num_vertices = len(adj_matrix)
    
    # compute the target Hamiltonian
    target = target_hamiltonian(adj_matrix)
    
    # model set up
    optimiser = qml.AdamOptimizer(stepsize=0.5)
    steps = 10
    num_layers = 2
    
    def qgscnn(weights, wires, L_matrix):
        for i in range(num_layers):
            qgscnn_layer(weights[i], wires, L_matrix)

        return qml.state()
    
    # Defines the new QNode
    qnode = qml.QNode(qgscnn, dev)
    
    def clustering_cost_fn(params):
        state = qnode(params, wires=range(num_vertices), L_matrix=L_matrix)
        print(state)
        print(target)
        loss = np.vdot(state, (target @ state))
        print(loss)
        return loss
    
    # to make life easier, kinetic weights are weights[layer][1], and coupling weights are weights[layer][0]
    weights = np.random.randint(-20, 20, size=(num_layers, 1, adj_matrix.shape[1]))/50
    init = copy.copy(weights)

    # Executes the optimization method
    iterations = 0

    for i in range(0, steps):
        print(f"Iteration {i}")
        weights = optimiser.step(clustering_cost_fn, weights)

    return qgrnn_params, init

In [97]:
quantum_clustering(weighted_graph_adj_matrix)

Iteration 0
Autograd ArrayBox with value [ 9.31515349e-01-1.40045034e-05j -1.39577906e-04+9.34632871e-02j
  4.22599613e-05-3.08694317e-01j  3.09727431e-02-4.21190008e-04j
  9.93777684e-05-1.31270908e-01j  1.31710235e-02-9.90462881e-04j
 -4.35017882e-02+2.99882153e-04j  2.98881879e-03-4.36473765e-03j
  1.74682680e-04-7.46806146e-02j  7.49305498e-03-1.74100016e-03j
 -2.47483644e-02+5.27122103e-04j  5.25363857e-03-2.48311903e-03j
 -1.05241337e-02+1.23957090e-03j  1.23543624e-02-1.05593551e-03j
 -3.74052664e-03+3.48758642e-03j -3.49925839e-04+3.72804990e-02j]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 16 is different from 4)